In [1]:
import pandas as pd
import numpy as np
import seaborn as sns; sns.set('talk')
import matplotlib.pyplot as plt
params = {'legend.fontsize': 'large',
          'figure.figsize': (16, 9),
         'axes.labelsize': 'large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'medium',
         'ytick.labelsize':'medium'}
plt.rcParams.update(params)
import json
from pandas.io.json import json_normalize
current_palette = sns.color_palette()
sns.palplot(current_palette)

In [13]:
in_file = pd.read_csv('msdnet-local.csv', index_col=0)
cols = ['exit', 'prediction', 'scores', 'target', 'overall time', 'correct', 'sample']
in_file.columns = cols
in_file.to_json('msdnet-local2.json', 'records')

In [14]:
d = in_file.to_dict('records')

In [43]:
for i in d:
    nums = [int(n) for n in i['prediction'][1:-1].split(',')]
    i['prediction']= nums


In [49]:
for i in d:
    nums = [float(n) for n in i['scores'][1:-1].split(',')]
    i['scores']= nums

In [54]:
for i in d:
    try:
        i['index_top5'] = i['prediction'].index(i['target'])
    except ValueError:
        i['index_top5'] = -1

In [55]:
d

[{'exit': 0,
  'prediction': [0, 60, 76, 92, 17],
  'scores': [0.9499828815460205,
   0.04576852172613144,
   0.0020117207895964384,
   0.00037493245326913893,
   0.0003097242733929306],
  'target': 0,
  'overall time': 41.754012000637886,
  'correct': True,
  'sample': 0,
  'index_top5': 0},
 {'exit': 1,
  'prediction': [0, 60, 92, 64, 62],
  'scores': [0.9837222695350647,
   0.01626780442893505,
   7.436334726662608e-06,
   1.2330323215792305e-06,
   5.032167678109545e-07],
  'target': 0,
  'overall time': 23.21549200041773,
  'correct': True,
  'sample': 0,
  'index_top5': 0},
 {'exit': 2,
  'prediction': [0, 60, 62, 91, 92],
  'scores': [0.9991095662117004,
   0.0008867254364304245,
   2.665262400114443e-06,
   9.082647807190369e-07,
   6.826865472930876e-08],
  'target': 0,
  'overall time': 14.376848000210884,
  'correct': True,
  'sample': 0,
  'index_top5': 0},
 {'exit': 3,
  'prediction': [0, 60, 92, 62, 91],
  'scores': [0.9995504021644592,
   0.00044944023829884827,
   1.288

In [56]:
import json
with open('msdnet-local3' +'.json', 'w') as f:
        json.dump(d, f)